In [ ]:
%pip install gym
%pip install matplotlib
%pip install pygame

In [18]:
import gym
import random
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Generate environment, reset it, then render it
env = gym.make("FrozenLake-v1", is_slippery=False, render_mode="ansi") # can do render_mode="human" for a pretty print, but will crash
state = env.reset()

print(env.render())


SFFF
FHFH
FFFH
HFFG

